# 02 · Cleaning & Feature Engineering
Normalize JSON, build transactions and RFM features.

In [ ]:
import pandas as pd, numpy as np, json, pathlib
DATA = pathlib.Path("../data")
products = pd.read_csv(DATA/"products.csv")
users = pd.read_csv(DATA/"users_raw.csv")
carts = pd.read_csv(DATA/"carts_raw.csv")

# Normalize users (handles JSON-like strings)
def safe_to_json(x):
    if isinstance(x, str):
        try:
            return json.loads(x.replace("'", '"'))
        except Exception:
            return None
    return x if isinstance(x, dict) else None

users_name = users["name"].apply(safe_to_json)
users_addr = users["address"].apply(safe_to_json)

users_clean = pd.DataFrame({
    "user_id": users["id"],
    "email": users.get("email", pd.Series([None]*len(users))),
    "username": users.get("username", pd.Series([None]*len(users))),
    "firstname": users_name.apply(lambda d: d.get("firstname") if isinstance(d, dict) else None),
    "lastname": users_name.apply(lambda d: d.get("lastname") if isinstance(d, dict) else None),
    "city": users_addr.apply(lambda d: d.get("city") if isinstance(d, dict) else None),
    "zipcode": users_addr.apply(lambda d: d.get("zipcode") if isinstance(d, dict) else None),
})

# Explode carts -> transaction lines
carts["date"] = pd.to_datetime(carts["date"], errors="coerce")
lines = []
for _, row in carts.iterrows():
    prods = safe_to_json(row["products"])
    if isinstance(prods, list):
        for it in prods:
            lines.append({
                "cart_id": row["id"],
                "user_id": row["userId"],
                "date": row["date"],
                "product_id": it.get("productId"),
                "quantity": it.get("quantity", 1)
            })
tx = pd.DataFrame(lines)

# Join price & compute revenue
tx = tx.merge(products[["id","title","category","price"]], left_on="product_id", right_on="id", how="left")
tx["line_revenue"] = tx["quantity"] * tx["price"]
tx = tx.drop(columns=["id"]).sort_values("date")

# RFM
rfm = (tx.groupby("user_id")
         .agg(last_purchase=("date","max"),
              frequency=("cart_id","nunique"),
              monetary=("line_revenue","sum"))
         .reset_index())
rfm["recency_days"] = (tx["date"].max() - rfm["last_purchase"]).dt.days
rfm = rfm.drop(columns=["last_purchase"])

# Save curated
tx.to_csv(DATA/"transactions.csv", index=False)
users_clean.to_csv(DATA/"users.csv", index=False)
rfm.to_csv(DATA/"user_rfm.csv", index=False)
print("Saved transactions.csv, users.csv, user_rfm.csv in", DATA)